In [1]:
import numpy as np
from copy import deepcopy
import random

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

/home/hhchung/dyngraph-uda/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred.evaluate import Evaluator

In [6]:
import sys
sys.path.append("..")
from dataset import temp_partition_arxiv

In [7]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')

In [8]:
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Model Structure ##

In [9]:
class TwoLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        return x

class ThreeLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv3(x, edge_index)
        return x
    

    
class MLPHead(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x = self.linear1(x)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        x = self.linear2(x)
        # x = F.softmax(x, dim=1)
        return x

In [14]:
def train(encoder, mlp, optimizer, data):
    encoder.train()
    mlp.train()
    
    out = F.log_softmax(mlp(encoder(data.x, data.edge_index)), dim=1)
    source_label_distribution = (torch.sum(F.one_hot(data.y, num_classes=40), dim=0) / data.y.shape[0])
    class_weights = 1.0 / source_label_distribution
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask], weight=class_weights)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()
    
@torch.no_grad()
def test(encoder, mlp, data, evaluator):
    encoder.eval()
    mlp.eval()
    source_label_distribution = (torch.sum(F.one_hot(data.y, num_classes=40), dim=0) / data.y.shape[0])
    class_weights = 1.0 / source_label_distribution
    out = F.log_softmax(mlp(encoder(data.x, data.edge_index)), dim=1)
    val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask], weight=class_weights).item()
    y_pred = out.argmax(dim=-1, keepdim=True)
    val_acc = evaluator.eval({
        'y_true': data.y[data.val_mask].unsqueeze(1),
        'y_pred': y_pred[data.val_mask],
    })['acc']
    
    return val_loss, val_acc

## Load Data ##

In [11]:
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')
data = dataset[0]
data.edge_index = to_undirected(data.edge_index, data.num_nodes) # mimicking barlow twins repo

## Data Partition ##

* Train: 0-2011
* Val: 2012
* Test:
** 2013-2014 (then adapt 2012-2013 adapt-val 2014)
** 2015-2016 (then adapt 2014-2015 adapt-val 2016)
** 2017-2018 (then adapt 2016-2017 adapt-val 2018)
** 2019-2020

## Source Training Stage ##

* Train: 0-2011
* Val: 2012

In [16]:
feat_dim = data.x.shape[1]
class_dim = data.y
hidden_dim = 128
emb_dim = 256
encoder = ThreeLayerGraphSAGE(feat_dim, hidden_dim, emb_dim)
mlp = MLPHead(emb_dim, emb_dim // 4, 40)

optimizer = torch.optim.Adam(list(encoder.parameters()) + list(mlp.parameters()), lr=1e-3)
epochs = 2000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
mlp = mlp.to(device)

In [13]:
data_2012_2013 = temp_partition_arxiv(data, year_bound=[-1,2012,2013], proportion=1.0)
data_2012_2013 = data_2012_2013.to(device)

In [17]:
best_acc = 0
best_encoder = None
best_mlp = None
evaluator = Evaluator(name='ogbn-arxiv')
for e in range(1, epochs + 1):
    train_loss = train(encoder, mlp, optimizer, data_2012_2013)
    val_loss, val_acc = test(encoder, mlp, data_2012_2013, evaluator)
    print(f"Epoch:{e}/{epochs} Train Loss:{round(train_loss,4)} Val Loss:{round(val_loss,4)} Val Acc:{round(val_acc, 4)}")
    if val_acc > best_acc:
        best_acc = val_acc
        best_encoder = deepcopy(encoder)
        best_mlp = deepcopy(mlp)

Epoch:1/2000 Train Loss:3.6904 Val Loss:3.6992 Val Acc:0.0149
Epoch:2/2000 Train Loss:3.678 Val Loss:3.7029 Val Acc:0.0152
Epoch:3/2000 Train Loss:3.6673 Val Loss:3.7067 Val Acc:0.0191
Epoch:4/2000 Train Loss:3.6575 Val Loss:3.7071 Val Acc:0.0272
Epoch:5/2000 Train Loss:3.6465 Val Loss:3.6992 Val Acc:0.039
Epoch:6/2000 Train Loss:3.6299 Val Loss:3.6894 Val Acc:0.0556
Epoch:7/2000 Train Loss:3.6138 Val Loss:3.6727 Val Acc:0.067
Epoch:8/2000 Train Loss:3.5942 Val Loss:3.6557 Val Acc:0.0968
Epoch:9/2000 Train Loss:3.5648 Val Loss:3.6324 Val Acc:0.115
Epoch:10/2000 Train Loss:3.5351 Val Loss:3.6069 Val Acc:0.1229
Epoch:11/2000 Train Loss:3.4953 Val Loss:3.5798 Val Acc:0.1277
Epoch:12/2000 Train Loss:3.4563 Val Loss:3.5475 Val Acc:0.1293
Epoch:13/2000 Train Loss:3.4137 Val Loss:3.5108 Val Acc:0.129
Epoch:14/2000 Train Loss:3.3629 Val Loss:3.4776 Val Acc:0.1394
Epoch:15/2000 Train Loss:3.3183 Val Loss:3.4429 Val Acc:0.1501
Epoch:16/2000 Train Loss:3.2754 Val Loss:3.4117 Val Acc:0.1657
Epoch:

Epoch:133/2000 Train Loss:1.6117 Val Loss:2.373 Val Acc:0.4099
Epoch:134/2000 Train Loss:1.6 Val Loss:2.336 Val Acc:0.4025
Epoch:135/2000 Train Loss:1.5937 Val Loss:2.3482 Val Acc:0.4028
Epoch:136/2000 Train Loss:1.5812 Val Loss:2.3486 Val Acc:0.405
Epoch:137/2000 Train Loss:1.5903 Val Loss:2.3433 Val Acc:0.4005
Epoch:138/2000 Train Loss:1.5834 Val Loss:2.3782 Val Acc:0.405
Epoch:139/2000 Train Loss:1.5885 Val Loss:2.3657 Val Acc:0.412
Epoch:140/2000 Train Loss:1.5768 Val Loss:2.3294 Val Acc:0.4154
Epoch:141/2000 Train Loss:1.5833 Val Loss:2.3772 Val Acc:0.3967
Epoch:142/2000 Train Loss:1.5736 Val Loss:2.3328 Val Acc:0.4034
Epoch:143/2000 Train Loss:1.5687 Val Loss:2.3259 Val Acc:0.4093
Epoch:144/2000 Train Loss:1.5537 Val Loss:2.3492 Val Acc:0.4106
Epoch:145/2000 Train Loss:1.5521 Val Loss:2.3152 Val Acc:0.4124
Epoch:146/2000 Train Loss:1.5441 Val Loss:2.3778 Val Acc:0.4089
Epoch:147/2000 Train Loss:1.5531 Val Loss:2.3584 Val Acc:0.4132
Epoch:148/2000 Train Loss:1.5346 Val Loss:2.307 

Epoch:267/2000 Train Loss:1.2503 Val Loss:2.4755 Val Acc:0.4309
Epoch:268/2000 Train Loss:1.2426 Val Loss:2.5138 Val Acc:0.4373
Epoch:269/2000 Train Loss:1.249 Val Loss:2.4646 Val Acc:0.4424
Epoch:270/2000 Train Loss:1.2511 Val Loss:2.4975 Val Acc:0.4421
Epoch:271/2000 Train Loss:1.2571 Val Loss:2.4739 Val Acc:0.4284
Epoch:272/2000 Train Loss:1.2273 Val Loss:2.5173 Val Acc:0.4371
Epoch:273/2000 Train Loss:1.2387 Val Loss:2.5119 Val Acc:0.4362
Epoch:274/2000 Train Loss:1.2339 Val Loss:2.5027 Val Acc:0.4317
Epoch:275/2000 Train Loss:1.2279 Val Loss:2.5086 Val Acc:0.4384
Epoch:276/2000 Train Loss:1.2185 Val Loss:2.4885 Val Acc:0.4381
Epoch:277/2000 Train Loss:1.2379 Val Loss:2.4836 Val Acc:0.4406
Epoch:278/2000 Train Loss:1.2333 Val Loss:2.4714 Val Acc:0.4342
Epoch:279/2000 Train Loss:1.2202 Val Loss:2.5234 Val Acc:0.4415
Epoch:280/2000 Train Loss:1.2186 Val Loss:2.6226 Val Acc:0.443
Epoch:281/2000 Train Loss:1.2254 Val Loss:2.5747 Val Acc:0.4319
Epoch:282/2000 Train Loss:1.2207 Val Loss:

Epoch:406/2000 Train Loss:1.0846 Val Loss:2.7316 Val Acc:0.4488
Epoch:407/2000 Train Loss:1.0742 Val Loss:2.834 Val Acc:0.4441
Epoch:408/2000 Train Loss:1.0736 Val Loss:2.8553 Val Acc:0.4595
Epoch:409/2000 Train Loss:1.0779 Val Loss:2.869 Val Acc:0.4573
Epoch:410/2000 Train Loss:1.0704 Val Loss:2.8578 Val Acc:0.4503
Epoch:411/2000 Train Loss:1.0633 Val Loss:2.9032 Val Acc:0.4451
Epoch:412/2000 Train Loss:1.0653 Val Loss:2.7449 Val Acc:0.4573
Epoch:413/2000 Train Loss:1.0656 Val Loss:2.7768 Val Acc:0.4482
Epoch:414/2000 Train Loss:1.0589 Val Loss:2.831 Val Acc:0.4469
Epoch:415/2000 Train Loss:1.0697 Val Loss:2.7929 Val Acc:0.4479
Epoch:416/2000 Train Loss:1.0676 Val Loss:2.8509 Val Acc:0.4519
Epoch:417/2000 Train Loss:1.0609 Val Loss:2.9024 Val Acc:0.4539
Epoch:418/2000 Train Loss:1.0692 Val Loss:2.9266 Val Acc:0.4561
Epoch:419/2000 Train Loss:1.0567 Val Loss:2.9181 Val Acc:0.455
Epoch:420/2000 Train Loss:1.0637 Val Loss:2.9651 Val Acc:0.4553
Epoch:421/2000 Train Loss:1.0601 Val Loss:2.

Epoch:555/2000 Train Loss:0.9519 Val Loss:3.0552 Val Acc:0.4583
Epoch:556/2000 Train Loss:0.9738 Val Loss:3.0608 Val Acc:0.4581
Epoch:557/2000 Train Loss:0.9604 Val Loss:3.3043 Val Acc:0.4494
Epoch:558/2000 Train Loss:0.9526 Val Loss:3.1778 Val Acc:0.4608
Epoch:559/2000 Train Loss:0.9585 Val Loss:3.3829 Val Acc:0.4662
Epoch:560/2000 Train Loss:0.9625 Val Loss:3.2477 Val Acc:0.4726
Epoch:561/2000 Train Loss:0.9514 Val Loss:3.3017 Val Acc:0.4632
Epoch:562/2000 Train Loss:0.9547 Val Loss:3.1436 Val Acc:0.4564
Epoch:563/2000 Train Loss:0.9499 Val Loss:3.1654 Val Acc:0.4598
Epoch:564/2000 Train Loss:0.9534 Val Loss:3.2649 Val Acc:0.462
Epoch:565/2000 Train Loss:0.9551 Val Loss:3.2094 Val Acc:0.4646
Epoch:566/2000 Train Loss:0.9462 Val Loss:3.2202 Val Acc:0.4623
Epoch:567/2000 Train Loss:0.9449 Val Loss:3.3217 Val Acc:0.4702
Epoch:568/2000 Train Loss:0.939 Val Loss:3.3087 Val Acc:0.4639
Epoch:569/2000 Train Loss:0.964 Val Loss:3.1812 Val Acc:0.453
Epoch:570/2000 Train Loss:0.95 Val Loss:3.08

...

Epoch:1382/2000 Train Loss:0.7067 Val Loss:4.9916 Val Acc:0.4659
Epoch:1383/2000 Train Loss:0.6961 Val Loss:4.8821 Val Acc:0.4713
Epoch:1384/2000 Train Loss:0.7086 Val Loss:4.9821 Val Acc:0.4735
Epoch:1385/2000 Train Loss:0.6986 Val Loss:5.2843 Val Acc:0.4802
Epoch:1386/2000 Train Loss:0.7172 Val Loss:4.9368 Val Acc:0.4729
Epoch:1387/2000 Train Loss:0.7086 Val Loss:4.9284 Val Acc:0.4738
Epoch:1388/2000 Train Loss:0.7043 Val Loss:4.8852 Val Acc:0.4785
Epoch:1389/2000 Train Loss:0.7005 Val Loss:4.8377 Val Acc:0.476
Epoch:1390/2000 Train Loss:0.704 Val Loss:4.7431 Val Acc:0.4821
Epoch:1391/2000 Train Loss:0.7142 Val Loss:5.1812 Val Acc:0.4713
Epoch:1392/2000 Train Loss:0.7095 Val Loss:5.022 Val Acc:0.4758
Epoch:1393/2000 Train Loss:0.6998 Val Loss:4.7727 Val Acc:0.4727
Epoch:1394/2000 Train Loss:0.7033 Val Loss:4.9604 Val Acc:0.4706
Epoch:1395/2000 Train Loss:0.698 Val Loss:5.0729 Val Acc:0.478
Epoch:1396/2000 Train Loss:0.704 Val Loss:4.8645 Val Acc:0.4727
Epoch:1397/2000 Train Loss:0.70

Epoch:1514/2000 Train Loss:0.6674 Val Loss:4.9097 Val Acc:0.4662
Epoch:1515/2000 Train Loss:0.6917 Val Loss:5.1815 Val Acc:0.4704
Epoch:1516/2000 Train Loss:0.6849 Val Loss:5.0641 Val Acc:0.4743
Epoch:1517/2000 Train Loss:0.6885 Val Loss:5.144 Val Acc:0.4811
Epoch:1518/2000 Train Loss:0.676 Val Loss:5.1629 Val Acc:0.4819
Epoch:1519/2000 Train Loss:0.7044 Val Loss:5.0865 Val Acc:0.4735
Epoch:1520/2000 Train Loss:0.6879 Val Loss:4.9914 Val Acc:0.4724
Epoch:1521/2000 Train Loss:0.6826 Val Loss:4.9152 Val Acc:0.4651
Epoch:1522/2000 Train Loss:0.6959 Val Loss:5.0015 Val Acc:0.474
Epoch:1523/2000 Train Loss:0.691 Val Loss:5.0581 Val Acc:0.4827
Epoch:1524/2000 Train Loss:0.6732 Val Loss:5.4038 Val Acc:0.4793
Epoch:1525/2000 Train Loss:0.7009 Val Loss:5.3071 Val Acc:0.4723
Epoch:1526/2000 Train Loss:0.6922 Val Loss:5.1081 Val Acc:0.469
Epoch:1527/2000 Train Loss:0.6952 Val Loss:4.9318 Val Acc:0.4738
Epoch:1528/2000 Train Loss:0.6899 Val Loss:5.0823 Val Acc:0.4744
Epoch:1529/2000 Train Loss:0.6

Epoch:1647/2000 Train Loss:0.6678 Val Loss:5.0053 Val Acc:0.4783
Epoch:1648/2000 Train Loss:0.679 Val Loss:5.1479 Val Acc:0.4789
Epoch:1649/2000 Train Loss:0.6725 Val Loss:5.4518 Val Acc:0.481
Epoch:1650/2000 Train Loss:0.6574 Val Loss:5.3376 Val Acc:0.4732
Epoch:1651/2000 Train Loss:0.6708 Val Loss:5.527 Val Acc:0.4761
Epoch:1652/2000 Train Loss:0.6644 Val Loss:5.4598 Val Acc:0.4802
Epoch:1653/2000 Train Loss:0.6792 Val Loss:5.2402 Val Acc:0.4679
Epoch:1654/2000 Train Loss:0.6616 Val Loss:5.1756 Val Acc:0.474
Epoch:1655/2000 Train Loss:0.6709 Val Loss:5.0401 Val Acc:0.4766
Epoch:1656/2000 Train Loss:0.6616 Val Loss:5.5236 Val Acc:0.4828
Epoch:1657/2000 Train Loss:0.6775 Val Loss:5.2563 Val Acc:0.481
Epoch:1658/2000 Train Loss:0.6701 Val Loss:5.3629 Val Acc:0.478
Epoch:1659/2000 Train Loss:0.6656 Val Loss:5.5533 Val Acc:0.4839
Epoch:1660/2000 Train Loss:0.6734 Val Loss:5.4066 Val Acc:0.481
Epoch:1661/2000 Train Loss:0.6589 Val Loss:5.1738 Val Acc:0.4805
Epoch:1662/2000 Train Loss:0.671

Epoch:1785/2000 Train Loss:0.6542 Val Loss:5.3676 Val Acc:0.478
Epoch:1786/2000 Train Loss:0.672 Val Loss:5.4391 Val Acc:0.476
Epoch:1787/2000 Train Loss:0.661 Val Loss:5.2358 Val Acc:0.4695
Epoch:1788/2000 Train Loss:0.6595 Val Loss:5.2873 Val Acc:0.4761
Epoch:1789/2000 Train Loss:0.6727 Val Loss:5.5181 Val Acc:0.4785
Epoch:1790/2000 Train Loss:0.6616 Val Loss:5.3755 Val Acc:0.4718
Epoch:1791/2000 Train Loss:0.6583 Val Loss:5.6384 Val Acc:0.4752
Epoch:1792/2000 Train Loss:0.6533 Val Loss:5.3874 Val Acc:0.4771
Epoch:1793/2000 Train Loss:0.6604 Val Loss:5.5279 Val Acc:0.4768
Epoch:1794/2000 Train Loss:0.6614 Val Loss:5.8084 Val Acc:0.4788
Epoch:1795/2000 Train Loss:0.6607 Val Loss:5.2145 Val Acc:0.478
Epoch:1796/2000 Train Loss:0.6488 Val Loss:5.5686 Val Acc:0.4749
Epoch:1797/2000 Train Loss:0.6517 Val Loss:5.591 Val Acc:0.4808
Epoch:1798/2000 Train Loss:0.6478 Val Loss:5.2527 Val Acc:0.4791
Epoch:1799/2000 Train Loss:0.659 Val Loss:5.5099 Val Acc:0.4805
Epoch:1800/2000 Train Loss:0.657

Epoch:1918/2000 Train Loss:0.6422 Val Loss:5.5585 Val Acc:0.4811
Epoch:1919/2000 Train Loss:0.6429 Val Loss:5.5641 Val Acc:0.4741
Epoch:1920/2000 Train Loss:0.6497 Val Loss:5.7931 Val Acc:0.4847
Epoch:1921/2000 Train Loss:0.6484 Val Loss:5.6179 Val Acc:0.4816
Epoch:1922/2000 Train Loss:0.6393 Val Loss:5.6093 Val Acc:0.4814
Epoch:1923/2000 Train Loss:0.6334 Val Loss:5.5465 Val Acc:0.4858
Epoch:1924/2000 Train Loss:0.6428 Val Loss:5.4917 Val Acc:0.4817
Epoch:1925/2000 Train Loss:0.6328 Val Loss:5.5513 Val Acc:0.4808
Epoch:1926/2000 Train Loss:0.6488 Val Loss:5.5793 Val Acc:0.4766
Epoch:1927/2000 Train Loss:0.6499 Val Loss:5.6305 Val Acc:0.4769
Epoch:1928/2000 Train Loss:0.6321 Val Loss:5.7528 Val Acc:0.4827
Epoch:1929/2000 Train Loss:0.6375 Val Loss:5.6444 Val Acc:0.4881
Epoch:1930/2000 Train Loss:0.6493 Val Loss:5.6756 Val Acc:0.4789
Epoch:1931/2000 Train Loss:0.6316 Val Loss:5.5144 Val Acc:0.4807
Epoch:1932/2000 Train Loss:0.6427 Val Loss:5.8018 Val Acc:0.488
Epoch:1933/2000 Train Loss